##任务 3：在 Amazon SageMaker 中部署模型

在此任务中，您将使用 Amazon SageMaker 部署经过训练的机器学习模型来获得预测。首先，将模型构件导出到 Amazon S3 存储桶。然后，从模型构件创建 Sagemaker 模型对象。接下来，通过指定 Sagemaker 模型、要转换的数据的 S3 存储桶位置以及用于存储结果的 S3 存储桶位置来创建批量转换作业。然后，启动批量转换作业并监控其进度。最后，批量转换作业完成后，从指定的 S3 存储桶中检索结果。结果将包含经过训练的模型对输入数据所做的推理。

**注意：**如果您在完成挑战时遇到任何问题，可在 Notebook 底部的<a href="#appendix" target="_self">**附录**</a>部分找到解决方案。

###任务 3.1：设置环境

这里的基本设置代码可以帮助您入门。请先查看并运行这些单元格，以安装程序包并创建变量。

In [ ]:
#import libraries and set variable values

import sys
import boto3
import os
import sagemaker
import time

sess = sagemaker.Session()

bucket = sess.default_bucket()
prefix = "lab3/batch-transform"
bucket_path = "s3://{}".format(bucket)

role = sagemaker.get_execution_role()

region = boto3.Session().region_name


###任务 3.2：创建模型

在此任务中，您将获取预训练模型并将其上传到 S3。然后，在 SageMaker 中创建模型。接下来，使用 SageMaker 批量转换作业部署该模型。

####要求和配置

- 使用 **S3.Client.upload_file** API 将模型构件和测试数据上传到 S3 存储桶。

有关使用 **upload_file** 方法的更多信息，请参阅 [将文件上传到 S3 对象] (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/upload_file.html#upload-file)。

In [ ]:
#将模型上传到 Amazon S3 存储桶
from sagemaker.session import Session
from sagemaker.image_uris import retrieve

sagemaker = boto3.client('sagemaker')
s3_client = boto3.client("s3")

#add_upload-model_code_here


#add_upload-dataset_code_here


**注意：**有关如何将模型构件和测试数据上传到 S3 存储桶的详细代码，请参阅*附录*部分中的<a href="#task3-2-1" target="_self">**将模型和测试数据上传到 S3（任务 3.2）**</a>。

<a id="task3-2-continue"></a>

接下来，从上述模型构件在 SageMaker 中创建模型对象。

####要求和配置

- 使用 **SageMaker.Client.create_model** API 在 SageMaker 中创建模型。
- 在 Request（请求）中，请使用以下选项：
    - ModelName='telemarketing-model'
    - ExecutionRoleArn=role (The IAM role create earlier that SageMaker uses)
    - 对于主容器，请使用以下选项：
        - Image= XGBoost container image URI (retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1'))
        - ModelDataUrl=model_path (The S3 path where the model artifacts are uploaded earlier) 

有关使用 **SageMaker.Client.create_model** 方法的更多信息，请参阅 [在 SageMaker 中创建模型] (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_model.html)。

In [ ]:
sagemaker = boto3.client('sagemaker')

#add_create-model_code_here

**注意：**有关如何创建模型的详细代码，请参阅*附录*部分中的<a href="#task3-2-2" target="_self">**在 SageMaker 中创建模型（任务 3.2）**</a>。

<a id="task3-3"></a>

###任务 3.3：创建批量转换器

在此任务中，您将创建批量转换器。

有关使用 **Transfomer API** 的更多信息，请参阅 [使用 Transformer API] (https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer)。

####要求和配置

- 使用 **sagemaker.transformer.Transformer** API 在 SageMaker 中初始化转换器。
- 在 Parameters（参数）部分中，请使用以下选项：
    - model_name="telemarketing-model"
    - instance_type="ml.m4.xlarge"
    - instance_count=1
    - assemble_with="Line"
    - strategy="MultiRecord"
    - accept="text/csv"
    - output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)

In [7]:
from sagemaker.transformer import Transformer

#add_create-batch-transformer_code_here
transformer = Transformer(
    
)

**注意：**有关如何创建转换器对象的详细代码，请参阅*附录*部分中的<a href="#task3-3-1" target="_self">**创建批量转换器对象（任务 3.3）**</a>。

<a id="task3-4"></a>

###任务 3.4：运行批量转换作业

在此任务中，您将运行转换作业。您将传递训练数据以从批量转换器获得预测。

有关使用 **Transformer.transform** API 的更多信息，请参阅 [使用 Transform API] (https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer.transform)。

####要求和配置

- 使用 **Transformer.transform** API 在 SageMaker 中初始化转换器。
- 在 Parameters（参数）部分中，请使用以下选项：
    - batch_path（Amazon S3 之前创建的输入数据位置）
    - content_type="text/csv"
    - split_type="Line"
    - join_source="Input"

In [ ]:
transformer.transform(
 #add_run-batch-transform-job_code_here   
)

transformer.wait()

**注意：**有关如何运行转换作业的详细代码，请参阅*附录*部分中的<a href="#task3-4-1" target="_self">**运行批量转换作业（任务 3.4）**</a>。

<a id="task3-5"></a>

###任务 3.5：验证存储在 S3 存储桶中的输出

在此任务中，您将在 S3 中提取批量转换作业的输出。应该会显示预测列表。

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
!head test_data.csv.out

恭喜！您已使用 Amazon SageMaker 成功部署了经过训练的机器学习模型来获得预测。

###清理

您已完成此 Notebook。要进入本实验的下一部分，请执行以下操作：

- 关闭此 Notebook 文件。
- 返回至实验会话并继续**总结**部分。

##其他资源

- [部署模型进行推理](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)
- [使用 SageMaker API](https://sagemaker.readthedocs.io/en/stable/api/inference/index.html)

<a name="appendix" id="appendix"></a>

##附录


<a name="task3-2-1" id="task3-2-1"></a>

###将模型和测试数据上传到 S3（任务 3.2）

In [ ]:
#将模型上传到 Amazon S3 存储桶
from sagemaker.session import Session
from sagemaker.image_uris import retrieve

#session = boto3.Session()
sagemaker = boto3.client('sagemaker')
s3_client = boto3.client("s3")

#upload-model
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")  
model_path = f"s3://{bucket}/{prefix}/models/model.tar.gz"
print(model_path)

#upload-dataset
s3_client.upload_file(Filename="test_data.csv", Bucket=bucket, Key=f"{prefix}/test_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket}/{prefix}/test_data.csv"
print(batch_path)

要继续本实验，请返回至<a href="#task3-2-continue" target="_self">任务 3.2</a>。

<a name="task3-2-2" id="task3-2-2"></a>

###在 SageMaker 中创建模型（任务 3.2）

In [ ]:
sagemaker = boto3.client('sagemaker')

#检索容器映像
image = retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1')

primary_container = {"Image": image, "ModelDataUrl": model_path}

#设置模型
create_model_response = sagemaker.create_model(
    ModelName='telemarketing-model', ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

要继续本实验，请返回至<a href="#task3-3" target="_self">任务 3.3</a>。

<a name="task3-3-1" id="task3-3-1"></a>

###创建批量转换器对象（任务 3.3）

In [ ]:
from sagemaker.transformer import Transformer

#create-batch-transformer
transformer = Transformer(
    model_name="telemarketing-model",
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)
)

要继续本实验，请返回至<a href="#task3-4" target="_self">任务 3.4</a>。

<a name="task3-4-1" id="task3-4-1"></a>

###运行批量转换作业（任务 3.4）


In [ ]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")

transformer.wait()

要继续本实验，请返回至<a href="#task3-5" target="_self">任务 3.5</a>。